### OCI Data Science - Useful Tips
<details>
<summary><font size="2">Check for Public Internet Access</font></summary>

```python
import requests
response = requests.get("https://oracle.com")
assert response.status_code==200, "Internet connection failed"
```
</details>
<details>
<summary><font size="2">Helpful Documentation </font></summary>
<ul><li><a href="https://docs.cloud.oracle.com/en-us/iaas/data-science/using/data-science.htm">Data Science Service Documentation</a></li>
<li><a href="https://docs.cloud.oracle.com/iaas/tools/ads-sdk/latest/index.html">ADS documentation</a></li>
</ul>
</details>
<details>
<summary><font size="2">Typical Cell Imports and Settings for ADS</font></summary>

```python
%load_ext autoreload
%autoreload 2
%matplotlib inline

import warnings
warnings.filterwarnings('ignore')

import logging
logging.basicConfig(format='%(levelname)s:%(message)s', level=logging.ERROR)

import ads
from ads.dataset.factory import DatasetFactory
from ads.automl.provider import OracleAutoMLProvider
from ads.automl.driver import AutoML
from ads.evaluations.evaluator import ADSEvaluator
from ads.common.data import ADSData
from ads.explanations.explainer import ADSExplainer
from ads.explanations.mlx_global_explainer import MLXGlobalExplainer
from ads.explanations.mlx_local_explainer import MLXLocalExplainer
from ads.catalog.model import ModelCatalog
from ads.common.model_artifact import ModelArtifact
```
</details>
<details>
<summary><font size="2">Useful Environment Variables</font></summary>

```python
import os
print(os.environ["NB_SESSION_COMPARTMENT_OCID"])
print(os.environ["PROJECT_OCID"])
print(os.environ["USER_OCID"])
print(os.environ["TENANCY_OCID"])
print(os.environ["NB_REGION"])
```
</details>

In [ ]:
##Langchain deployment

In [7]:
import tempfile
import ads
from ads.model.generic_model import GenericModel
from config import CONDA_PACK_PATH, LOG_GROUP_ID, LANGCHAIN_MODEL_ACCESS_LOG_LOG_ID, LANGCHAIN_MODEL_PREDICT_LOG_LOG_ID

 
ads.set_auth("resource_principal")
 
langchain_model = GenericModel( artifact_dir="langchain_model",estimator=None, serialize=False)
langchain_model.summary_status()

Actions Needed
Step      Status        Details                                                           
initiate  Done          Initiated the model                                               
prepare() Available     Generated runtime.yaml                                            
                        Generated score.py                                                
                        Serialized model                                                  
                        Populated metadata(Custom, Taxonomy and Provenance)               
verify()  Not Available Local tested .predict from score.py                               
save()    Not Available Conducted Introspect Test                                         
                        Uploaded artifact to model catalog                                
deploy()  UNKNOWN       Deployed the model                                                
predict() Not Available Called deployment predict endpoint

In [8]:
#Copy required python scripts to artifact dir 
#cp config.py config_private.py oci_utils.py oracle_vector_db.py langchain_model/
langchain_model.prepare(
        inference_conda_env=CONDA_PACK_PATH,
        inference_python_version = "3.9",
        model_file_name="test",
        score_py_uri= "langchain_model_score.py",
        force_overwrite=True
     )

INFO:ADS:To auto-extract taxonomy metadata the model must be provided. Supported models: keras, lightgbm, pytorch, sklearn, tensorflow, pyspark, and xgboost.


algorithm: null
artifact_dir:
  /home/datascience/langchain_model:
  - - oracle_vector_db.py
    - config.py
    - config_private.py
    - runtime.yaml
    - test_json_output.json
    - score.py
    - .model-ignore
    - oci_utils.py
framework: null
model_deployment_id: null
model_id: null

In [ ]:
op=langchain_model.verify('Can you please give some overview on Oracle Database 23.4?')


In [10]:
print(op)

{'prediction': ' Oracle Database 23.4 is a new release of Oracle Database that introduces a new VECTOR data type for advanced AI/ML operations. This data type is currently available as a homogeneous array of either 8-bit signed integers or of 32-bit or 64-bit floating point numbers. The python-oracledb extension module for Python is a database driver for high-performance Oracle Database applications written in Python 3. This release also includes support for Oracle Database AI Vector Search, which is available in the Free (23.4.0.24.02) Limited Availability release. VECTOR columns in Oracle Database can be created as VECTOR(<vectorDimensions>, <vectorDimensionFormat>), where the attributes are the number of dimensions for the vector data and the storage format for each dimension value in the vector. The python-oracledb driver has direct access to Oracle Database through its default Thin mode, and an optional Thick mode can be enabled at runtime using the Oracle Client libraries for con

In [11]:
model_id = langchain_model.save(display_name="langchain-model")

['oracle_vector_db.py', 'config.py', 'config_private.py', 'runtime.yaml', 'test_json_output.json', 'score.py', '.model-ignore', 'oci_utils.py']


loop1:   0%|          | 0/4 [00:00<?, ?it/s]

In [12]:
deploy = langchain_model.deploy(
    display_name="Langchain Model Deployment",
    deployment_log_group_id = LOG_GROUP_ID,
    deployment_access_log_id = LANGCHAIN_MODEL_ACCESS_LOG_LOG_ID,
    deployment_predict_log_id = LANGCHAIN_MODEL_PREDICT_LOG_LOG_ID,
    environment_variables={"CRYPTOGRAPHY_OPENSSL_NO_LEGACY":"1"},
    deployment_instance_shape="VM.Standard2.4",
)

loop1:   0%|          | 0/6 [00:00<?, ?it/s]

In [22]:
deploy = langchain_model.deploy(
    display_name="Langchain Model Deployment",
    deployment_log_group_id = LOG_GROUP_ID,
    deployment_access_log_id = LANGCHAIN_MODEL_ACCESS_LOG_LOG_ID,
    deployment_predict_log_id = LANGCHAIN_MODEL_PREDICT_LOG_LOG_ID,
    environment_variables={"CRYPTOGRAPHY_OPENSSL_NO_LEGACY":"1"},
    deployment_instance_shape="VM.Standard2.4",
    deployment_instance_subnet_id="ocid1.subnet.oc1.<ocid>",
)

loop1:   0%|          | 0/6 [00:00<?, ?it/s]